In [161]:
#convert dicom CT image to jpg format and downsample to 128 * 128
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from typing import Tuple
from PIL import Image
import glob
from pathlib import Path
import os
import keras
from keras import layers
from keras.datasets import mnist
import matplotlib.pyplot as plt

In [162]:
#load data
data_path =  '/Users/jasperzzz/Library/CloudStorage/OneDrive-UniversityofToronto/BiostatPhD/Year1/Fall&Y_courses/CSC2529ComputImage/data/'
ct256path = data_path + "processed4people/256/"
full_list = sorted([f for f in os.listdir(ct256path ) if f.endswith('Full.png')])
low_list = sorted([f for f in os.listdir(ct256path ) if f.endswith('Low.png')])

img_dirs = os.listdir(ct256path )
full = []
low = []

for name in full_list:
    img_path = os.path.join( ct256path, name ) 
    image = Image.open( img_path ).convert( 'L' ) 
    full.append(np.asarray(image))
    
for name in low_list:
    img_path = os.path.join( ct256path, name ) 
    image = Image.open( img_path ).convert( 'L' ) 
    low.append(np.asarray(image))
#store iamges as array
full = np.asarray(full)
low = np.asarray(low)

In [139]:
low_list[1]

'N005_img256_1-02._Low.png'

In [129]:
name = 'N005_img256_1-02._Low.jpg'
img_path = os.path.join( ct256path, name ) 
image = Image.open( img_path ).convert( 'L' ) 
iarray = np.asarray(image)


array([[[1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 7, 7, 6],
        [1, 1, 1, ..., 5, 6, 5],
        [0, 1, 1, ..., 4, 3, 2]],

       [[2, 2, 1, ..., 1, 2, 1],
        [2, 2, 2, ..., 2, 1, 2],
        [2, 1, 2, ..., 1, 1, 2],
        ...,
        [1, 2, 2, ..., 3, 3, 2],
        [1, 1, 1, ..., 3, 3, 2],
        [1, 1, 1, ..., 3, 3, 3]],

       [[2, 1, 1, ..., 1, 2, 2],
        [1, 2, 2, ..., 2, 2, 1],
        [1, 1, 2, ..., 2, 2, 1],
        ...,
        [1, 1, 1, ..., 4, 5, 4],
        [1, 1, 2, ..., 5, 5, 5],
        [2, 2, 2, ..., 4, 3, 3]],

       ...,

       [[2, 2, 2, ..., 1, 1, 1],
        [2, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 1, 1, 1],
        ...,
        [1, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 1, ..., 2, 2, 2]],

       [[2, 2, 2, ..., 1, 1, 1],
        [1, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 1, 1, 2],
        ...,
        [2, 2, 2, ..., 

In [168]:
x_train = full
x_test = low

In [41]:
dimension = x_train.shape[1]
print(dimension)

256


In [165]:
print(np.max(x_test[1,:,:,0]))
np.min(x_test[1,:,:,0])

9.274685e-13


0.0

In [169]:
norm_factor = 255.
x_train = x_train.astype('float32')/norm_factor
x_test = x_test.astype('float32')/norm_factor
x_train = np.reshape(x_train, (len(x_train), dimension, dimension, 1))
x_test = np.reshape(x_test, (len(x_test), dimension, dimension, 1))

In [170]:
x_train

array([[[[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.00392157],
         [0.00392157],
         [0.        ]],

        [[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.00392157],
         [0.00392157],
         [0.00392157]],

        [[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.00392157],
         [0.00392157],
         [0.00392157]],

        ...,

        [[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.02352941],
         [0.02352941],
         [0.02352941]],

        [[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.02352941],
         [0.01960784],
         [0.01960784]],

        [[0.00392157],
         [0.00392157],
         [0.00392157],
         ...,
         [0.01568628],
         [0.01176471],
         [0.00784314]]],


       [[[0.00784314],
         [0.00784314],
         [0.00

In [171]:
from sklearn.model_selection import train_test_split
x_train8, x_train2, x_test8, x_test2 = train_test_split(x_train, x_test, test_size=0.2, random_state=42)

In [172]:
input_img = keras.Input(shape = (dimension, dimension, 1))

In [173]:
x = layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(input_img)
x = layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(x)
x = layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
encoded = layers.MaxPooling2D(pool_size = (2, 2), padding = 'same')(x)

In [174]:
x = layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(encoded)
x = layers.UpSampling2D(size = (2, 2))(x)
x = layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.UpSampling2D(size = (2, 2))(x)
decoded = layers.Conv2D(filters = 1, kernel_size = (3, 3), activation = 'sigmoid', padding = 'same')(x)

In [175]:
autoencoder = keras.Model(input_img, decoded)

In [176]:
#autoencoder.summary()

In [177]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [178]:
validation_split = 0.8
history = autoencoder.fit(x_train8, x_test8, epochs = 128, batch_size = 20, shuffle = True, validation_split = validation_split)

Epoch 1/128


2022-11-28 13:11:36.484099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 0.6907

2022-11-28 13:11:37.248872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 776ms/step - loss: 0.6904 - val_loss: 0.6791
Epoch 2/128
2/2 [==============================] - 0s 319ms/step - loss: 0.6759 - val_loss: 0.6641
Epoch 3/128
2/2 [==============================] - 0s 330ms/step - loss: 0.6576 - val_loss: 0.6516
Epoch 4/128
2/2 [==============================] - 0s 322ms/step - loss: 0.6384 - val_loss: 0.6282
Epoch 5/128
2/2 [==============================] - 0s 326ms/step - loss: 0.6112 - val_loss: 0.5848
Epoch 6/128
2/2 [==============================] - 0s 337ms/step - loss: 0.5678 - val_loss: 0.5247
Epoch 7/128
2/2 [==============================] - 0s 334ms/step - loss: 0.5076 - val_loss: 0.4478
Epoch 8/128
2/2 [==============================] - 0s 334ms/step - loss: 0.4335 - val_loss: 0.3656
Epoch 9/128
2/2 [==============================] - 0s 327ms/step - loss: 0.3531 - val_loss: 0.3069
Epoch 10/128
2/2 [==============================] - 0s 338ms/step - loss: 0.2977 - val_loss: 0.2898
Epoch 11/128
2/2 [===

2/2 [==============================] - 0s 336ms/step - loss: 0.2489 - val_loss: 0.2529
Epoch 84/128
2/2 [==============================] - 0s 323ms/step - loss: 0.2465 - val_loss: 0.2529
Epoch 85/128
2/2 [==============================] - 0s 328ms/step - loss: 0.2525 - val_loss: 0.2527
Epoch 86/128
2/2 [==============================] - 0s 331ms/step - loss: 0.2502 - val_loss: 0.2526
Epoch 87/128
2/2 [==============================] - 0s 324ms/step - loss: 0.2460 - val_loss: 0.2529
Epoch 88/128
2/2 [==============================] - 0s 331ms/step - loss: 0.2464 - val_loss: 0.2527
Epoch 89/128
2/2 [==============================] - 0s 327ms/step - loss: 0.2467 - val_loss: 0.2525
Epoch 90/128
2/2 [==============================] - 1s 337ms/step - loss: 0.2496 - val_loss: 0.2524
Epoch 91/128
2/2 [==============================] - 0s 334ms/step - loss: 0.2460 - val_loss: 0.2525
Epoch 92/128
2/2 [==============================] - 0s 331ms/step - loss: 0.2480 - val_loss: 0.2525
Epoch 93/128


In [179]:
denoised_2 = autoencoder.predict(x_train2)

2022-11-28 13:12:49.289596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [180]:
denoised_2.shape

(35, 256, 256, 1)

In [181]:
loss = autoencoder.evaluate(x_train2, x_test2, batch_size = 20)

2/2 [==============================] - 0s 66ms/step - loss: 0.2575


In [182]:
from skimage import metrics


In [183]:
# calculate denoised PSNR
PSNR_linear = metrics.peak_signal_noise_ratio(x_test2, denoised_2)
print(PSNR_linear)

30.539277398689173


In [184]:
# calculate PSNR
PSNR_linear = metrics.peak_signal_noise_ratio(x_test2, x_train2)
print(PSNR_linear)

52.18691503065663


In [185]:
metrics.mean_squared_error(x_test2, denoised_2)

0.0008832268439809461

In [186]:
metrics.mean_squared_error(x_test2, x_train2)

6.0437779092021544e-06

In [202]:
img = Image.fromarray(x_train2[10,:,:,0], 'RGB')
img.show()

In [200]:
np.max(x_train2[10,:,:,0])

0.96862745

In [187]:
Image.fromarray(x_train[1,:,:,],'L')

ValueError: Too many dimensions: 3 > 2.

9.274685e-13